# Movielens 영화 추천 실습

## 데이터 준비

In [3]:
import os
import pandas as pd
import numpy as np
rating_file_path = os.getenv('HOME')+'/aiffel/recommendata_iu/data/ml-1m/ratings.dat'

In [4]:
os.getcwd()

'/home/jin/workplace/first-repository'

In [5]:
#win 버전 메타데이터 위치 변화
#rating_file_path = 'D:\\AI\\first-repository/ratings.dat'

In [6]:
# win 버전 제목메타데이터 변화
#movie_file_path = 'D:\\AI\\first-repository/movies.dat'

In [7]:
pd.read_csv(rating_file_path)

,1::1193::5::978300760
0,1::661::3::978302109
1,1::914::3::978301968
2,1::3408::4::978300275
3,1::2355::5::978824291
4,1::1197::3::978302268
...,...
1000203,6040::1091::1::956716541
1000204,6040::1094::5::956704887
1000205,6040::562::5::956704746
1000206,6040::1096::4::956715648


In [8]:
# 원본 파일 내용을 :: 기준으로 나눠야 할 것 같다.
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings=pd.read_csv(rating_file_path, sep="::", names=ratings_cols, engine='python')

In [9]:
original_data_size = len(ratings)

ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [10]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [11]:
# 3점 이상만 남김 - 부정적인건 추천할 필요없으니까
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'original_data_size:{original_data_size}, filtered_data_size:{filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / original_data_size:.2%}')

original_data_size:1000209, filtered_data_size:836478
Ratio of Remaining Data is 83.63%


In [12]:
# rating column의 이름을 count로 변경
ratings.rename(columns={'rating':'count'}, inplace=True)

In [13]:
ratings.keys()

Index(['user_id', 'movie_id', 'count', 'timestamp'], dtype='object')

In [14]:
# # 영화제목을 보기위해 메타데이터를 읽어옴
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 

In [15]:
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [17]:
# 시간정보 drop
ratings = ratings.loc[:,['user_id','movie_id','count']]

In [18]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


## 데이터 정보와 전처리

* movies :  영화데이터 3883개, 중복없이 3883개
* ratings : 평점데이터 836478개, 중복없이 836478개 

In [19]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  3883 non-null   int64 
 1   title     3883 non-null   object
 2   genre     3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


* 유니크한 영화갯수

In [20]:
movies.movie_id.nunique()

3883

* 유니크한 사용자수

In [21]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836478 entries, 0 to 1000208
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   user_id   836478 non-null  int64
 1   movie_id  836478 non-null  int64
 2   count     836478 non-null  int64
dtypes: int64(3)
memory usage: 25.5 MB


In [22]:
ratings.user_id.nunique()

6039

## 내 데이터 추가

* 내가 좋아하는 영화 5개

In [23]:
import re

In [24]:
movies['title'].loc[163]

'Die Hard: With a Vengeance (1995)'

In [25]:
# 제목을 편히 찾기위해 제목칼럼을 소문자로 바꿈
movies['title'] = movies.title.str.lower()

In [26]:
my_movie = ['gladiator','die hard','mask','terminator 2', 'home alone']

In [27]:
word = re.compile('gladiator|die hard|mask|terminator 2|home alone') # or =|

In [28]:
a = 1
b = 1
a is b

True

In [29]:
# items = key,value 반환
for i,name in movies['title'].items(): 
    pattern = word.search(name)
    if pattern is not None:
        print(movies.loc[i].values)
        

[165 'die hard: with a vengeance (1995)' 'Action|Thriller']
[367 'mask, the (1994)' 'Comedy|Crime|Fantasy']
[586 'home alone (1990)' "Children's|Comedy"]
[589 'terminator 2: judgment day (1991)' 'Action|Sci-Fi|Thriller']
[1036 'die hard (1988)' 'Action|Thriller']
[1370 'die hard 2 (1990)' 'Action|Thriller']
[1707 'home alone 3 (1997)' "Children's|Comedy"]
[1801 'man in the iron mask, the (1998)' 'Action|Drama|Romance']
[2006 'mask of zorro, the (1998)' 'Action|Adventure|Romance']
[2609 'king of masks, the (bian lian) (1996)' 'Drama']
[2625 'black mask (hak hap) (1996)' 'Action']
[2953 'home alone 2: lost in new york (1992)' "Children's|Comedy"]
[3213 'batman: mask of the phantasm (1993)' "Animation|Children's"]
[3578 'gladiator (2000)' 'Action|Drama']


In [32]:
# 165
# die hard: with a vengeance (1995)
# 367
# mask, the (1994)
# 586
# terminator 2: judgment day (1991)
# 3578
# gladiator (2000)
# 586
# home alone (1990)

In [33]:
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


In [34]:
ratings.head()

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


* 내 정보 입력

In [35]:
# 위에서 찾은 영화ID, 임의의 평점 입력
my_list = pd.DataFrame({'user_id':[6041]*5, 'movie_id':[165,367,589,586,3578], 'count':[5,5,5,5,4]})

In [36]:
my_list

,user_id,movie_id,count
0,6041,165,5
1,6041,367,5
2,6041,589,5
3,6041,586,5
4,6041,3578,4


In [37]:
# 그냥 추가하는게 아닌 해당ID 있는지 확인후 추가
# 해당DF의 user_id column에 6041이 1개라도 있느냐?
if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings  = ratings.append(my_list)

print(ratings.shape)    
ratings[ratings['user_id'] == 6041]

(836483, 3)


,user_id,movie_id,count
0,6041,165,5
1,6041,367,5
2,6041,589,5
3,6041,586,5
4,6041,3578,4


In [38]:
movies_1 = pd.merge(ratings, movies, on='movie_id')

movies_1.sort_values('user_id', ascending = True, inplace=True)
movies_1.reset_index(inplace=True, drop=True)
movies_1

movies_1.isin({'user_id':[6041]}) # 마지막 5개의 user_id가 6041이다.

,user_id,movie_id,count,title,genre
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
836478,True,False,False,False,False
836479,True,False,False,False,False
836480,True,False,False,False,False
836481,True,False,False,False,False


In [39]:
movies_1 = pd.merge(ratings, movies, on='movie_id')

In [40]:
movies_1.sort_values('user_id', ascending = True, inplace=True)
movies_1.reset_index(inplace=True, drop=True)
movies_1

,user_id,movie_id,count,title,genre
0,1,1193,5,one flew over the cuckoo's nest (1975),Drama
1,1,2294,4,antz (1998),Animation|Children's
2,1,3186,4,"girl, interrupted (1999)",Drama
3,1,1566,4,hercules (1997),Adventure|Animation|Children's|Comedy|Musical
4,1,588,4,aladdin (1992),Animation|Children's|Comedy|Musical
...,...,...,...,...,...
836478,6041,367,5,"mask, the (1994)",Comedy|Crime|Fantasy
836479,6041,165,5,die hard: with a vengeance (1995),Action|Thriller
836480,6041,589,5,terminator 2: judgment day (1991),Action|Sci-Fi|Thriller
836481,6041,3578,4,gladiator (2000),Action|Drama


* 가장 인기있는 영화 30개(인기순)

In [41]:
movie_count = ratings.groupby('movie_id')['count'].sum()
m_count = movie_count.sort_values(ascending=False).head(30)

In [42]:
m_count

movie_id
2858    14449
260     13178
1196    12648
2028    11348
1210    11303
1198    11179
593     11096
2571    10903
2762    10703
589     10518
608     10465
527     10317
110     10125
318     10085
1270    10081
858      9965
1197     9866
480      9601
1617     9515
2396     9503
1580     9057
296      9053
2997     8964
1265     8727
1097     8696
356      8679
1240     8571
1        8475
2716     8282
1214     8244
Name: count, dtype: int64

In [39]:
[movies[movies['movie_id'] == i].values.tolist() for i in m_count.index]

[[[2858, 'american beauty (1999)', 'Comedy|Drama']],
 [[260,
   'star wars: episode iv - a new hope (1977)',
   'Action|Adventure|Fantasy|Sci-Fi']],
 [[1196,
   'star wars: episode v - the empire strikes back (1980)',
   'Action|Adventure|Drama|Sci-Fi|War']],
 [[2028, 'saving private ryan (1998)', 'Action|Drama|War']],
 [[1210,
   'star wars: episode vi - return of the jedi (1983)',
   'Action|Adventure|Romance|Sci-Fi|War']],
 [[1198, 'raiders of the lost ark (1981)', 'Action|Adventure']],
 [[593, 'silence of the lambs, the (1991)', 'Drama|Thriller']],
 [[2571, 'matrix, the (1999)', 'Action|Sci-Fi|Thriller']],
 [[2762, 'sixth sense, the (1999)', 'Thriller']],
 [[589, 'terminator 2: judgment day (1991)', 'Action|Sci-Fi|Thriller']],
 [[608, 'fargo (1996)', 'Crime|Drama|Thriller']],
 [[527, "schindler's list (1993)", 'Drama|War']],
 [[110, 'braveheart (1995)', 'Action|Drama|War']],
 [[318, 'shawshank redemption, the (1994)', 'Drama']],
 [[1270, 'back to the future (1985)', 'Comedy|Sci-Fi'

* 상위 영화 30개 표현 간단히

In [43]:
pivot = pd.pivot_table(movies_1,index='title', values='count', aggfunc=np.sum)
pivot.sort_values(by='count', ascending = False).head(30)

,count
title,
american beauty (1999),14449
star wars: episode iv - a new hope (1977),13178
star wars: episode v - the empire strikes back (1980),12648
saving private ryan (1998),11348
star wars: episode vi - return of the jedi (1983),11303
raiders of the lost ark (1981),11179
"silence of the lambs, the (1991)",11096
"matrix, the (1999)",10903
"sixth sense, the (1999)",10703


## 번외 : 장르별 컬럼 생성

In [44]:
cc=movies['genre']
cc

0        Animation|Children's|Comedy
1       Adventure|Children's|Fantasy
2                     Comedy|Romance
3                       Comedy|Drama
4                             Comedy
                    ...             
3878                          Comedy
3879                           Drama
3880                           Drama
3881                           Drama
3882                  Drama|Thriller
Name: genre, Length: 3883, dtype: object

In [45]:
movie1 = movies.copy()

In [46]:
f = lambda x: x.split('|')

In [47]:
movie1['genre']=cc.apply(f)

In [48]:
movie1

,movie_id,title,genre
0,1,toy story (1995),"[Animation, Children's, Comedy]"
1,2,jumanji (1995),"[Adventure, Children's, Fantasy]"
2,3,grumpier old men (1995),"[Comedy, Romance]"
3,4,waiting to exhale (1995),"[Comedy, Drama]"
4,5,father of the bride part ii (1995),[Comedy]
...,...,...,...
3878,3948,meet the parents (2000),[Comedy]
3879,3949,requiem for a dream (2000),[Drama]
3880,3950,tigerland (2000),[Drama]
3881,3951,two family house (2000),[Drama]


In [46]:
for i, g in enumerate(cc):
    gs = g.lower().split('|')
    for _ in gs:
        movie1.loc[i, _] = 1

In [47]:
movie1

,movie_id,title,genre,animation,children's,comedy,adventure,fantasy,romance,drama,...,crime,thriller,horror,sci-fi,documentary,war,musical,mystery,film-noir,western
0,1,toy story (1995),"[Animation, Children's, Comedy]",1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,jumanji (1995),"[Adventure, Children's, Fantasy]",NaN,1.0,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,grumpier old men (1995),"[Comedy, Romance]",NaN,NaN,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,waiting to exhale (1995),"[Comedy, Drama]",NaN,NaN,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,father of the bride part ii (1995),[Comedy],NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,meet the parents (2000),[Comedy],NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3879,3949,requiem for a dream (2000),[Drama],NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3880,3950,tigerland (2000),[Drama],NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3881,3951,two family house (2000),[Drama],NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
ratings.shape

(836483, 3)

In [49]:
movies_1.isin({'user_id':[6041]})

,user_id,movie_id,count,title,genre
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
836478,True,False,False,False,False
836479,True,False,False,False,False
836480,True,False,False,False,False
836481,True,False,False,False,False


* CSR Matrix 만들기

In [50]:
from scipy.sparse import csr_matrix
import numpy as np
from implicit.als import AlternatingLeastSquares

In [51]:
# matrix 만들때 빈칸이 있으면 안되므로 빈칸을 뒤의 번호로 당기면  가장 길이가 긴값이 기준이 되므로

m_user = max(movies_1.user_id)
m_movie =max(movies_1.movie_id)

In [52]:
movies_1['count']

0         5
1         4
2         4
3         4
4         4
         ..
836478    5
836479    5
836480    5
836481    4
836482    5
Name: count, Length: 836483, dtype: int64

In [53]:
#
m_user, m_movie

(6041, 3952)

In [54]:
# 영화데이터, 유저데이터 기반한 점수를 matrix화
csr_data = csr_matrix((movies_1['count'],(movies_1['user_id'], movies_1['movie_id'])), shape=(m_user+1, m_movie+1)) #size+1
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [55]:
# factor 차원, iterations 반복수
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

In [56]:
csr_data_transpose = csr_data.T

In [57]:
als_model.fit(csr_data_transpose)

* 모델이 예측한 나의 선호도

In [58]:
jin_vec, gladi = als_model.user_factors[6041], als_model.item_factors[3578]

In [59]:
np.dot(jin_vec, gladi)

0.5031698

* 특정 영화1에 대한 모델예측한 나의 선호도ㅡ

In [60]:
zz = movies_1[movies_1['movie_id']==588]

In [61]:
zz[:1]

,user_id,movie_id,count,title,genre
4,1,588,4,aladdin (1992),Animation|Children's|Comedy|Musical


In [62]:
jin_vec, ano_mov = als_model.user_factors[6041], als_model.item_factors[588]

In [64]:
np.dot(jin_vec, ano_mov)

0.121357486

In [65]:
# 알라딘 좋아하는데 선호도 매우 낮게 나옴, 영화데이터에 애니메이션이 없기때문일까

* 내가 좋아하는 영화와 비슷한 영화 추천

In [66]:
# filter_already_liked_items : 이미 평가한 아이템은 제외
movie_recommend = als_model.recommend(6041, csr_data, N=20, filter_already_liked_items=True)

In [67]:
movie_recommend

[(2571, 0.33198845),
 (1240, 0.27834862),
 (480, 0.26635703),
 (1370, 0.23904705),
 (648, 0.23779929),
 (2762, 0.23289384),
 (3247, 0.22839813),
 (3753, 0.2254899),
 (500, 0.22443718),
 (1610, 0.22187507),
 (457, 0.21200085),
 (2628, 0.2096647),
 (3623, 0.20800154),
 (317, 0.20750937),
 (2916, 0.2049279),
 (110, 0.19350483),
 (1036, 0.19207028),
 (3793, 0.19115959),
 (1580, 0.1880982),
 (3489, 0.18126066)]

In [68]:
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


* 추천된 영화의 제목

In [69]:
for i in movie_recommend:
    
    vv = movies[movies['movie_id']==i[0]]
    print(vv['title'].values)

['matrix, the (1999)']
['terminator, the (1984)']
['jurassic park (1993)']
['die hard 2 (1990)']
['mission: impossible (1996)']
['sixth sense, the (1999)']
['sister act (1992)']
['patriot, the (2000)']
['mrs. doubtfire (1993)']
['hunt for red october, the (1990)']
['fugitive, the (1993)']
['star wars: episode i - the phantom menace (1999)']
['mission: impossible 2 (2000)']
['santa clause, the (1994)']
['total recall (1990)']
['braveheart (1995)']
['die hard (1988)']
['x-men (2000)']
['men in black (1997)']
['hook (1991)']


In [70]:
movies[movies['movie_id']==3489]

,movie_id,title,genre
3420,3489,hook (1991),Adventure|Fantasy


In [71]:
movie_recommend[0][0]

2571

In [72]:
# explain메소드 : 기록을 남긴 데이터중 이 추천에 기여한 정도 확인 가능
gladiator = 3578
explain = als_model.explain(6041, csr_data, itemid=gladiator)

In [73]:
explain

(0.4960991742220333,
 [(3578, 0.4949621522252301),
  (589, 0.00736587952299255),
  (586, 0.006525895705143255),
  (367, 0.0033087632131177944),
  (165, -0.016063516444450393)],
 (array([[ 0.66838555,  0.15244573,  0.10429911, ...,  0.10289953,
           0.08744936,  0.09749471],
         [ 0.10189252,  0.6360832 ,  0.08174462, ...,  0.132812  ,
           0.095602  ,  0.05860878],
         [ 0.06971202,  0.06789633,  0.60467742, ...,  0.08533085,
           0.08320311,  0.05814114],
         ...,
         [ 0.06877656,  0.10016608,  0.07318663, ...,  0.57281881,
           0.00229535, -0.01068614],
         [ 0.05844989,  0.07414211,  0.06724688, ...,  0.0724723 ,
           0.55913325,  0.02660637],
         [ 0.06516406,  0.05214271,  0.0501162 , ...,  0.04717655,
           0.07980195,  0.55514236]]),
  False))

In [74]:
[(movies[movies['movie_id']==i[0]].values, i) for i in explain[1]]

[(array([[3578, 'gladiator (2000)', 'Action|Drama']], dtype=object),
  (3578, 0.4949621522252301)),
 (array([[589, 'terminator 2: judgment day (1991)',
          'Action|Sci-Fi|Thriller']], dtype=object),
  (589, 0.00736587952299255)),
 (array([[586, 'home alone (1990)', "Children's|Comedy"]], dtype=object),
  (586, 0.006525895705143255)),
 (array([[367, 'mask, the (1994)', 'Comedy|Crime|Fantasy']], dtype=object),
  (367, 0.0033087632131177944)),
 (array([[165, 'die hard: with a vengeance (1995)', 'Action|Thriller']],
        dtype=object),
  (165, -0.016063516444450393))]

In [ ]:
# array 정리하고 싶다..